# DIC - shear zone field analysis

In [ ]:
%matplotlib widget

In [ ]:
import matplotlib as plt
import numpy as np
import os

# System independent file management

In [ ]:
from os.path import expanduser, join

Get the user directlry and locate the data relative to it

In [ ]:
home_dir = expanduser('~')
data_dir = join(home_dir,'simdb','data','shear_zone','B1_TV1')
data_dir

Get the list of files within the directory and sort them alphabetically

In [ ]:
results = [join(data_dir,each) 
           for each in sorted(os.listdir(data_dir)) 
           if each.endswith('.csv')]
results

Verify - read a single file

In [ ]:
np.loadtxt(results[4], dtype=float, 
            skiprows=1, delimiter=',', usecols=(2,3), unpack=False)

# Read the displacements into a single two dimensional array

Read all files as array using `loadtxt` and put them into a list, than make an array from the list, show the shape. Denote the dimensionality of the array using the index names, i.e. `u_te`, with `t` denoting the time index and `e` data entry index 

In [ ]:
u_te = np.array([
    np.loadtxt(csv_file, dtype=float,  
               skiprows=1, delimiter=',', usecols=(4), unpack=False)
    for csv_file in results
], dtype=np.float_)
u_te.shape
u_te = np.hstack([u_te])

Reshape the array such that the last two indexes are the x and y displacements, the array indexes are then
`tpa`
where `p` denotes the point index and `a` the dimensional index

In [ ]:
n_t, n_e = u_te.shape # get the dimensions of the time and entry dimensions
u_tpa = u_te.reshape(n_t,-1,2)
u_tpa.shape

In [ ]:
u_tpa[-1,:34,1]

In [ ]:
add_point_tpa=np.zeros((5,1,2))
uu_tpa = np.concatenate([u_tpa, add_point_tpa],axis=1)
uu_tpa.shape

In [ ]:
n_cols, n_rows = 36, 24

In [ ]:
u_tjia = uu_tpa.reshape(n_t,n_rows, n_cols,2)
u_tija = np.einsum('tjia->tija', u_tjia)
u_tjia.shape, u_tija.shape

In [ ]:
#u1x_tija = u_tija[:,:-1,:,:]

In [ ]:
#u1_tija = u1x_tija[:,::4,14:,:]

In [ ]:
#u1_tija[-1,:,:,1].shape

# Generate the grid of initial positions

In [ ]:
d_x, d_y = 14, 14

In [ ]:
_, n_c, n_r, _ = u1_tija.shape 
#n_c, n_r = n_cols-1, n_rows
x_range = np.arange(n_c)[::-1] * d_x
y_range = np.arange(n_r) * d_y
y_ij, x_ij = np.meshgrid(y_range, x_range)

In [ ]:
X_aij = np.array([x_ij, y_ij])
X_ija = np.einsum('aij->ija', X_aij)
X_ija.shape

In [ ]:
#X_ija

# Convert to a FE mesh

# Evaluate a center of rotation from displacement data on a grid

Calculate the difference between the origin and the displacement at time $t$

$$
\Delta u_{tija} = u_{tija} - u_{0ija}
$$

Perpendicular vector to the movement of the point

$$
\perp u_{aij} = [\Delta u_{tij1}, -\Delta u_{tij0}]
$$

In [ ]:
u1_tija.shape

In [ ]:
delta_u_ija = u1_tija[-1] - u1_tija[-2]
#delta_u_ija = u1_tija[1] - u1_tija[0]
avg_a = np.average(delta_u_ija, axis=(0,1))
avg_a

In [ ]:
u_rot_ija = delta_u_ija - avg_a[np.newaxis,np.newaxis,:] 

In [ ]:
rot_Xu_ija = X_ija + u_rot_ija * 100
rot_vect_u_nija = np.array([X_ija, rot_Xu_ija])
rot_vect_u_anij = np.einsum('nija->anij', rot_vect_u_nija)
rot_vect_u_anp = rot_vect_u_anij.reshape(2,2,-1)

In [ ]:
#rot_vect_u_anp

In [ ]:
perp_u_aij = np.array([u_rot_ija[...,1], -u_rot_ija[...,0]])
perp_u_ija = np.einsum('aij->ija', perp_u_aij)
perp_Xu_ija = X_ija + perp_u_ija * 100
perp_vect_u_nija = np.array([X_ija, perp_Xu_ija])
perp_vect_u_anij = np.einsum('nija->anij', perp_vect_u_nija)
perp_vect_u_anp = perp_vect_u_anij.reshape(2,2,-1)
#perp_vect_u_anp

In [ ]:
import matplotlib.pylab as plt
fig, ax = plt.subplots(1,1)
ax.plot(*rot_vect_u_anp,color='blue');
ax.plot(*perp_vect_u_anp,color='green');
ax.axis('equal');

The point on the line can be expressed using a scalar $\eta_{p}$ 
$$
  x^{\mathrm{CoR}}_{pa} = x_{pa} + \eta_{p} (\perp\!u_{pa})
$$


$$
R = \sqrt{(x_{pa}^\mathrm{CoR} - x_{qa}^\mathrm{CoR}) (x_{pa}^\mathrm{CoR} - x_{qa}^\mathrm{CoR})}
$$

$$ (x_a^\mathrm{CoR} - x_{ija}) (x_a^\mathrm{CoR} - x_{ija})
$$

In [ ]:
X_pa = X_ija.reshape(-1,2)
perp_u_pa = perp_u_ija.reshape(-1,2)

In [ ]:
def get_x_cor_pa(eta_p):
    return X_pa + np.einsum('p,pa->pa', eta_p, perp_u_pa)

In [ ]:
def get_R(eta_p):
    x_cor_pa = get_x_cor_pa(eta_p)
    delta_x_cor_pqa = x_cor_pa[:,np.newaxis,:] - x_cor_pa[np.newaxis,:,:]
    R2 = np.einsum('pqa,pqa->',delta_x_cor_pqa,delta_x_cor_pqa)
    return np.sqrt(R2)

In [ ]:
eta0_p = np.zeros((X_pa.shape[0],))

In [ ]:
get_R(eta0_p)

In [ ]:
from scipy.optimize import minimize

In [ ]:
min_eta_p_sol = minimize(get_R, eta0_p, method='BFGS')
eta_p_sol = min_eta_p_sol.x

In [ ]:
x_cor_pa_sol = get_x_cor_pa(eta_p_sol)

In [ ]:
X_cor = np.average(x_cor_pa_sol,axis=0)
X_cor

In [ ]:
import matplotlib.pylab as plt
fig, ax = plt.subplots(1,1)
ax.plot(*rot_vect_u_anp,color='blue');
ax.plot(*perp_vect_u_anp,color='green');
ax.plot(*x_cor_pa_sol.T, 'o')
ax.plot([X_cor[0]], [X_cor[1]], 'o',color='red')
ax.axis('equal');

# TODO

- Implement the above functionality as a bu.Model with an interactive control of the visualization.
- Dimensions of the particular specimen - to plot the measured DIC field within the beam.
- Parameters - specifying the input file, the shape of the grid, the spacing.
- Slices for the further evaluation


# Concept of the strain evaluation